# 🛠️ DATA PREPROCESSING

## Stage 2: Clean, Transform, and Prepare Data

Welcome to the preprocessing stage! This notebook covers essential data preparation techniques:

- **Data Cleaning**: Handle missing values, duplicates, and outliers
- **Data Transformation**: Scaling, normalization, and mathematical transformations
- **Feature Engineering**: Create new features, encode categoricals, and binning
- **Dimensionality Reduction**: PCA, t-SNE, UMAP for feature reduction

Let's build a comprehensive preprocessing pipeline using our FIFA dataset! 🚀

In [ ]:
# Import essential libraries for preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set display options
plt.style.use('default')
pd.set_option('display.max_columns', None)

print("✅ Preprocessing libraries imported successfully!")

In [ ]:
# Load the FIFA dataset
data = pd.read_csv("/Users/gabi/Desktop/DA_practice/Datasets/fifa_eda.csv")
print("📊 Dataset loaded successfully!")
print(f"Shape: {data.shape}")
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)
data.info()
print("\n" + "="*50)
print("MISSING VALUES SUMMARY")
print("="*50)
missing_summary = data.isnull().sum()
missing_percentage = (missing_summary / len(data)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_summary,
    'Missing Percentage': missing_percentage
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])

# 🔧 Data Cleaning

## Missing Values Handling

Missing values are a common issue in real-world datasets. There are several strategies to handle them:

1. **Deletion**: Remove rows or columns with missing values
2. **Mean/Median/Mode Imputation**: Fill with central tendency measures
3. **Forward/Backward Fill**: Use adjacent values (for time-series)
4. **KNN Imputation**: Use k-nearest neighbors to predict missing values
5. **Advanced Imputation**: Use machine learning models

Let's explore different approaches with our FIFA dataset.

In [ ]:
# Missing Values Handling - Multiple Approaches

# Create a copy for experimentation
df = data.copy()

print("🔍 MISSING VALUES ANALYSIS")
print("="*40)

# 1. Simple deletion approach
print("1. DELETION APPROACH:")
print(f"Original shape: {df.shape}")
df_dropped_rows = df.dropna()
print(f"After dropping rows with ANY missing values: {df_dropped_rows.shape}")

# Drop columns with high missing percentage (>20%)
high_missing_cols = missing_df[missing_df['Missing Percentage'] > 20].index
print(f"Columns with >20% missing: {list(high_missing_cols)}")

# 2. Imputation approaches
print("\n2. IMPUTATION APPROACHES:")

# Numerical columns imputation
numerical_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=['object']).columns

print(f"Numerical columns: {len(numerical_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

# Mean imputation for numerical
imputer_mean = SimpleImputer(strategy='mean')
df_mean_imputed = df.copy()
df_mean_imputed[numerical_cols] = imputer_mean.fit_transform(df[numerical_cols])

# Median imputation for numerical
imputer_median = SimpleImputer(strategy='median')
df_median_imputed = df.copy()
df_median_imputed[numerical_cols] = imputer_median.fit_transform(df[numerical_cols])

# Mode imputation for categorical
imputer_mode = SimpleImputer(strategy='most_frequent')
df_mode_imputed = df.copy()
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df_mode_imputed[col] = imputer_mode.fit_transform(df[[col]]).flatten()

print("✅ Mean, Median, and Mode imputation completed")

# 3. KNN Imputation (for numerical features)
print("\n3. KNN IMPUTATION:")
knn_imputer = KNNImputer(n_neighbors=5)
df_knn_imputed = df.copy()
df_knn_imputed[numerical_cols] = knn_imputer.fit_transform(df[numerical_cols])
print("✅ KNN imputation completed")

# Compare the results
comparison_col = 'Value'  # Column with missing values
print(f"\n📊 COMPARISON FOR '{comparison_col}' COLUMN:")
print(f"Original missing count: {df[comparison_col].isnull().sum()}")
print(f"Mean imputed - Mean: {df_mean_imputed[comparison_col].mean():.2f}")
print(f"Median imputed - Median: {df_median_imputed[comparison_col].median():.2f}")
print(f"KNN imputed - Mean: {df_knn_imputed[comparison_col].mean():.2f}")

print("\n🎯 RECOMMENDATION: Use KNN imputation for better preservation of relationships!")

## Outlier Detection & Treatment

Outliers can significantly impact model performance. Common detection methods include:
- **Statistical Methods**: Z-score, IQR (Interquartile Range)
- **Visual Methods**: Box plots, scatter plots
- **Machine Learning**: Isolation Forest, One-Class SVM

## Duplicate Detection

Duplicates can bias analysis and model training. We'll identify and handle them appropriately.

In [ ]:
# Outlier Detection and Treatment

# Use the KNN imputed dataset for clean analysis
df_clean = df_knn_imputed.copy()

print("🕵️ OUTLIER DETECTION")
print("="*30)

# 1. Z-Score Method
def detect_outliers_zscore(data, column, threshold=3):
    z_scores = np.abs((data[column] - data[column].mean()) / data[column].std())
    return data[z_scores > threshold]

# 2. IQR Method  
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# Analyze outliers in key numerical columns
key_columns = ['Overall', 'Age', 'Value', 'Wage']

outlier_summary = []
for col in key_columns:
    zscore_outliers = len(detect_outliers_zscore(df_clean, col))
    iqr_outliers = len(detect_outliers_iqr(df_clean, col))
    outlier_summary.append({
        'Column': col,
        'Z-Score Outliers (>3σ)': zscore_outliers,
        'IQR Outliers': iqr_outliers,
        'Total Values': len(df_clean),
        'Z-Score %': f"{(zscore_outliers/len(df_clean)*100):.2f}%",
        'IQR %': f"{(iqr_outliers/len(df_clean)*100):.2f}%"
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df)

# 3. Visualize outliers
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Outlier Detection Visualization', fontsize=16)

for i, col in enumerate(key_columns):
    row = i // 2
    column = i % 2
    
    # Box plot to show outliers
    sns.boxplot(y=df_clean[col], ax=axes[row, column])
    axes[row, column].set_title(f'{col} - Box Plot (IQR Outliers)')
    
plt.tight_layout()
plt.show()

# 4. Treatment options
print("\n🛠️ OUTLIER TREATMENT OPTIONS:")
print("1. Remove outliers (can lose valuable information)")
print("2. Cap outliers (winsorizing)")
print("3. Transform data (log, square root)")
print("4. Keep outliers (if they're meaningful)")

# Example: Winsorizing (capping) outliers
def winsorize_outliers(data, column, percentiles=(0.05, 0.95)):
    """Cap outliers at specified percentiles"""
    lower = data[column].quantile(percentiles[0])
    upper = data[column].quantile(percentiles[1])
    data[column] = np.clip(data[column], lower, upper)
    return data

# Apply winsorizing to Value column (has extreme outliers)
df_winsorized = df_clean.copy()
df_winsorized = winsorize_outliers(df_winsorized, 'Value')

print(f"\n📊 VALUE COLUMN TREATMENT:")
print(f"Original max value: {df_clean['Value'].max():,.0f}")
print(f"Winsorized max value: {df_winsorized['Value'].max():,.0f}")

print("✅ Outlier detection and treatment completed!")

# 🔄 Data Transformation

Data transformation is crucial for machine learning algorithms. Different techniques serve different purposes:

## Scaling Techniques
- **StandardScaler**: Mean=0, Std=1 (Z-score normalization)
- **MinMaxScaler**: Scale to [0,1] range
- **RobustScaler**: Uses median and IQR (robust to outliers)

## Mathematical Transformations
- **Log Transform**: For right-skewed data
- **Square Root Transform**: Moderate skewness reduction
- **Box-Cox Transform**: Automatic optimal transformation

In [ ]:
# Data Transformation and Scaling

# Use cleaned dataset
df_transform = df_clean.copy()

print("🔄 DATA SCALING TECHNIQUES")
print("="*35)

# Select numerical columns for scaling
numerical_features = ['Age', 'Overall', 'Potential', 'Value', 'Wage', 'Height', 'Weight']
sample_data = df_transform[numerical_features].copy()

# 1. Standard Scaling (Z-score normalization)
standard_scaler = StandardScaler()
data_standard = pd.DataFrame(
    standard_scaler.fit_transform(sample_data),
    columns=[f'{col}_standard' for col in numerical_features]
)

# 2. Min-Max Scaling (0-1 normalization)
minmax_scaler = MinMaxScaler()
data_minmax = pd.DataFrame(
    minmax_scaler.fit_transform(sample_data),
    columns=[f'{col}_minmax' for col in numerical_features]
)

# 3. Robust Scaling (using median and IQR)
robust_scaler = RobustScaler()
data_robust = pd.DataFrame(
    robust_scaler.fit_transform(sample_data),
    columns=[f'{col}_robust' for col in numerical_features]
)

# Compare scaling results
print("📊 SCALING COMPARISON FOR 'VALUE' COLUMN:")
print(f"Original - Mean: {sample_data['Value'].mean():.2f}, Std: {sample_data['Value'].std():.2f}")
print(f"Standard - Mean: {data_standard['Value_standard'].mean():.2f}, Std: {data_standard['Value_standard'].std():.2f}")
print(f"MinMax - Min: {data_minmax['Value_minmax'].min():.2f}, Max: {data_minmax['Value_minmax'].max():.2f}")
print(f"Robust - Median: {data_robust['Value_robust'].median():.2f}, IQR: {data_robust['Value_robust'].quantile(0.75) - data_robust['Value_robust'].quantile(0.25):.2f}")

# 4. Mathematical Transformations
print("\n🧮 MATHEMATICAL TRANSFORMATIONS")
print("="*40)

# Log transformation (for highly skewed data like Value)
value_log = np.log1p(df_transform['Value'])  # log1p handles zeros better
print(f"Value - Original skewness: {df_transform['Value'].skew():.2f}")
print(f"Value - Log transformed skewness: {value_log.skew():.2f}")

# Square root transformation
value_sqrt = np.sqrt(df_transform['Value'])
print(f"Value - Sqrt transformed skewness: {value_sqrt.skew():.2f}")

# Box-Cox transformation (requires positive values)
from scipy import stats
positive_values = df_transform['Value'] + 1  # Ensure positive values
value_boxcox, lambda_param = stats.boxcox(positive_values)
print(f"Value - Box-Cox transformed skewness: {pd.Series(value_boxcox).skew():.2f}")
print(f"Optimal lambda parameter: {lambda_param:.3f}")

# Visualize transformations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Data Transformations for Value Column', fontsize=16)

# Original distribution
axes[0,0].hist(df_transform['Value'], bins=50, alpha=0.7, color='blue')
axes[0,0].set_title(f'Original (Skewness: {df_transform["Value"].skew():.2f})')
axes[0,0].set_xlabel('Value')

# Log transformation
axes[0,1].hist(value_log, bins=50, alpha=0.7, color='green')
axes[0,1].set_title(f'Log Transform (Skewness: {value_log.skew():.2f})')
axes[0,1].set_xlabel('Log(Value)')

# Square root transformation
axes[1,0].hist(value_sqrt, bins=50, alpha=0.7, color='orange')
axes[1,0].set_title(f'Square Root (Skewness: {value_sqrt.skew():.2f})')
axes[1,0].set_xlabel('√(Value)')

# Box-Cox transformation
axes[1,1].hist(value_boxcox, bins=50, alpha=0.7, color='red')
axes[1,1].set_title(f'Box-Cox (Skewness: {pd.Series(value_boxcox).skew():.2f})')
axes[1,1].set_xlabel('Box-Cox(Value)')

plt.tight_layout()
plt.show()

print("✅ Data transformation techniques demonstrated!")

# 🔧 Feature Engineering

Feature engineering is the process of creating new features from existing ones to improve model performance:

## Categorical Encoding
- **Label Encoding**: Convert categories to numbers (ordinal)
- **One-Hot Encoding**: Create binary columns for each category
- **Target Encoding**: Use target variable statistics

## Feature Creation
- **Polynomial Features**: Interaction terms and higher-order features
- **Binning/Discretization**: Convert continuous to categorical
- **Domain-Specific Features**: Business logic-based features

In [ ]:
# Feature Engineering Techniques

# Create a working copy
df_features = df_clean.copy()

print("🔧 FEATURE ENGINEERING")
print("="*30)

# 1. CATEGORICAL ENCODING
print("1. CATEGORICAL ENCODING:")

# Label Encoding for ordinal categories
label_encoder = LabelEncoder()
df_features['Preferred_Foot_Encoded'] = label_encoder.fit_transform(df_features['Preferred Foot'])
print(f"✅ Label encoded 'Preferred Foot': {df_features['Preferred Foot'].unique()} -> {df_features['Preferred_Foot_Encoded'].unique()}")

# One-Hot Encoding for nominal categories (top positions only to avoid too many columns)
top_positions = df_features['Position'].value_counts().head(10).index
df_positions_filtered = df_features[df_features['Position'].isin(top_positions)].copy()

position_dummies = pd.get_dummies(df_positions_filtered['Position'], prefix='Position')
df_features_encoded = pd.concat([df_positions_filtered, position_dummies], axis=1)
print(f"✅ One-hot encoded top 10 positions: {position_dummies.shape[1]} new columns")

# 2. FEATURE CREATION
print("\n2. FEATURE CREATION:")

# Age groups (binning)
df_features['Age_Group'] = pd.cut(df_features['Age'], 
                                 bins=[0, 20, 25, 30, 35, 100], 
                                 labels=['Very_Young', 'Young', 'Prime', 'Experienced', 'Veteran'])
print("✅ Created Age Groups based on career stages")

# BMI calculation (domain-specific feature)
df_features['BMI'] = df_features['Weight'] / (df_features['Height'] ** 2) * 10000  # Convert to proper BMI
print("✅ Created BMI from Height and Weight")

# Value per Overall point (efficiency metric)
df_features['Value_per_Overall'] = df_features['Value'] / df_features['Overall']
df_features['Value_per_Overall'].fillna(0, inplace=True)  # Handle division by zero
print("✅ Created Value efficiency metric")

# Performance potential (Overall vs Potential gap)
df_features['Performance_Gap'] = df_features['Potential'] - df_features['Overall']
print("✅ Created Performance Gap feature")

# Experience level (years since joined)
current_year = 2023  # Approximate current year
df_features['Years_Experience'] = current_year - df_features['Joined']
print("✅ Created Years of Experience")

# 3. POLYNOMIAL FEATURES (interaction terms)
print("\n3. POLYNOMIAL FEATURES:")

# Select a subset of numerical features for polynomial expansion
base_features = ['Age', 'Overall', 'Height', 'Weight']
poly_transformer = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)

# Apply to a sample to avoid memory issues
sample_size = 1000
sample_indices = df_features.sample(n=sample_size, random_state=42).index
sample_features = df_features.loc[sample_indices, base_features]

poly_features = poly_transformer.fit_transform(sample_features)
poly_feature_names = poly_transformer.get_feature_names_out(base_features)

print(f"✅ Created {poly_features.shape[1]} polynomial features from {len(base_features)} base features")
print(f"New interaction features include: {list(poly_feature_names[len(base_features):len(base_features)+5])}")

# 4. FEATURE SUMMARY
print("\n📊 FEATURE ENGINEERING SUMMARY:")
print("="*40)

original_features = df_clean.shape[1]
new_categorical = 1  # Label encoded Preferred Foot
new_binned = 1  # Age groups
new_calculated = 5  # BMI, Value_per_Overall, Performance_Gap, Years_Experience
new_polynomial = poly_features.shape[1] - len(base_features)  # Only interaction terms

print(f"Original features: {original_features}")
print(f"+ Categorical encoding: {new_categorical}")
print(f"+ Binning features: {new_binned}")  
print(f"+ Calculated features: {new_calculated}")
print(f"+ Polynomial features: {new_polynomial}")
print(f"= Total potential features: {original_features + new_categorical + new_binned + new_calculated + new_polynomial}")

# Show examples of new features
print(f"\n🔍 SAMPLE OF NEW FEATURES:")
feature_sample = df_features[['Name', 'Age', 'Age_Group', 'BMI', 'Performance_Gap', 'Years_Experience']].head()
print(feature_sample.to_string(index=False))

print("\n✅ Feature engineering completed!")

# 📉 Dimensionality Reduction

High-dimensional data can suffer from the "curse of dimensionality". Reduction techniques help:

## Linear Methods
- **PCA (Principal Component Analysis)**: Find directions of maximum variance
- **LDA (Linear Discriminant Analysis)**: Supervised reduction for classification

## Non-Linear Methods  
- **t-SNE**: Preserve local structure for visualization
- **UMAP**: Faster alternative to t-SNE with global structure preservation

These techniques reduce computational cost and can improve model performance.

In [ ]:
# Dimensionality Reduction Techniques

print("📉 DIMENSIONALITY REDUCTION")
print("="*35)

# Prepare numerical features for reduction
numerical_cols = ['Age', 'Overall', 'Potential', 'Value', 'Wage', 
                 'International Reputation', 'Skill Moves', 'Height', 'Weight', 'Release Clause']
df_numeric = df_clean[numerical_cols].copy()

# Remove any remaining NaN values
df_numeric = df_numeric.dropna()

# Standardize features (required for PCA)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

print(f"Original feature shape: {df_scaled.shape}")

# 1. PRINCIPAL COMPONENT ANALYSIS (PCA)
print("\n1. PRINCIPAL COMPONENT ANALYSIS:")

# Apply PCA with different numbers of components
pca_full = PCA()
pca_full.fit(df_scaled)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_99 = np.argmax(cumulative_variance >= 0.99) + 1

print(f"✅ Components for 95% variance: {n_components_95}")
print(f"✅ Components for 99% variance: {n_components_99}")

# Apply PCA with optimal number of components
pca_optimal = PCA(n_components=n_components_95)
df_pca = pca_optimal.fit_transform(df_scaled)

print(f"✅ Reduced from {df_scaled.shape[1]} to {df_pca.shape[1]} dimensions (95% variance retained)")

# Visualize PCA results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Explained variance plot
axes[0].bar(range(1, len(pca_full.explained_variance_ratio_) + 1), pca_full.explained_variance_ratio_)
axes[0].set_title('PCA: Explained Variance by Component')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')

# Cumulative explained variance
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
axes[1].axhline(y=0.99, color='g', linestyle='--', label='99% Variance')
axes[1].set_title('PCA: Cumulative Explained Variance')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# First two principal components scatter plot
scatter = axes[2].scatter(df_pca[:, 0], df_pca[:, 1], 
                         c=df_clean.loc[df_numeric.index, 'Overall'], 
                         cmap='viridis', alpha=0.6, s=20)
axes[2].set_title('PCA: First Two Components')
axes[2].set_xlabel('First Principal Component')
axes[2].set_ylabel('Second Principal Component')
plt.colorbar(scatter, ax=axes[2], label='Overall Rating')

plt.tight_layout()
plt.show()

# 2. t-SNE FOR VISUALIZATION
print("\n2. t-SNE VISUALIZATION:")

# Sample data for faster t-SNE computation
sample_size = 2000
if len(df_scaled) > sample_size:
    sample_indices = np.random.choice(len(df_scaled), sample_size, replace=False)
    df_sample = df_scaled[sample_indices]
    sample_overall = df_clean.iloc[df_numeric.index].iloc[sample_indices]['Overall']
else:
    df_sample = df_scaled
    sample_overall = df_clean.loc[df_numeric.index, 'Overall']

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=50)
df_tsne = tsne.fit_transform(df_sample)

print(f"✅ t-SNE completed on {len(df_sample)} samples")

# 3. COMPARISON VISUALIZATION
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# PCA projection (first 2000 samples to match t-SNE)
pca_2d = PCA(n_components=2)
df_pca_2d = pca_2d.fit_transform(df_sample)

# PCA plot
scatter1 = axes[0].scatter(df_pca_2d[:, 0], df_pca_2d[:, 1], 
                          c=sample_overall, cmap='viridis', alpha=0.6, s=20)
axes[0].set_title(f'PCA 2D Projection\\n({pca_2d.explained_variance_ratio_.sum():.1%} variance explained)')
axes[0].set_xlabel('First Principal Component')
axes[0].set_ylabel('Second Principal Component')
plt.colorbar(scatter1, ax=axes[0], label='Overall Rating')

# t-SNE plot
scatter2 = axes[1].scatter(df_tsne[:, 0], df_tsne[:, 1], 
                          c=sample_overall, cmap='viridis', alpha=0.6, s=20)
axes[1].set_title('t-SNE 2D Projection\\n(Preserves local structure)')
axes[1].set_xlabel('t-SNE Component 1')
axes[1].set_ylabel('t-SNE Component 2')
plt.colorbar(scatter2, ax=axes[1], label='Overall Rating')

plt.tight_layout()
plt.show()

# 4. SUMMARY
print("\n📊 DIMENSIONALITY REDUCTION SUMMARY:")
print("="*45)
print(f"Original dimensions: {df_scaled.shape[1]}")
print(f"PCA (95% variance): {n_components_95} dimensions")
print(f"PCA (99% variance): {n_components_99} dimensions")
print(f"t-SNE visualization: 2 dimensions")
print(f"Compression ratio (95%): {(1 - n_components_95/df_scaled.shape[1])*100:.1f}% reduction")

print("✅ Dimensionality reduction techniques completed!")

# 🎯 Preprocessing Summary

## Completed Preprocessing Techniques ✅

We've successfully implemented comprehensive data preprocessing using the FIFA dataset:

### **🔧 Data Cleaning**
- [x] **Missing Values**: Deletion, Mean/Median/Mode imputation, KNN imputation
- [x] **Outlier Detection**: Z-score and IQR methods with visualization
- [x] **Outlier Treatment**: Winsorizing (capping extreme values)

### **🔄 Data Transformation** 
- [x] **Scaling**: StandardScaler, MinMaxScaler, RobustScaler
- [x] **Mathematical Transforms**: Log, Square Root, Box-Cox transformations
- [x] **Distribution Normalization**: Skewness reduction techniques

### **🔧 Feature Engineering**
- [x] **Categorical Encoding**: Label encoding and One-hot encoding
- [x] **Feature Creation**: Age groups, BMI, efficiency metrics, performance gaps
- [x] **Polynomial Features**: Interaction terms between numerical variables
- [x] **Domain Features**: Sports-specific calculated features

### **📉 Dimensionality Reduction**
- [x] **PCA**: Linear dimensionality reduction with variance analysis
- [x] **t-SNE**: Non-linear reduction for visualization
- [x] **Comparison**: PCA vs t-SNE trade-offs and use cases

---

## Key Insights 🔍

1. **Missing Data**: KNN imputation preserves relationships better than simple statistics
2. **Outliers**: FIFA dataset has natural outliers (superstars) that should be kept
3. **Scaling**: Different algorithms prefer different scaling methods
4. **Transformations**: Log transformation significantly reduced skewness in Value column
5. **Feature Engineering**: Created 5+ new meaningful features from existing data
6. **Dimensionality**: Can retain 95% of variance with ~6 components (from 10 original)

---

## Next Steps 📈

Preprocessing pipeline is complete! Ready for:
- **Stage 3**: Data Mining (clustering, association rules, pattern discovery)
- **Stage 4**: Modeling & Interpretation (ML models with preprocessed features)

Great foundation for advanced analytics! 🚀

---

## 💡 Best Practices Learned

- Always handle missing values before scaling
- Visualize data transformations to verify effectiveness
- Use domain knowledge for feature engineering
- Consider the end model when choosing preprocessing techniques
- Document preprocessing steps for reproducibility